In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
model_dir = '/kuacc/users/mali18/dicomogan/logs/_510_face_splice2022-11-01T02-55-15'
img_root = '/userfiles/abond19/video_datasets/RAVDESS/RAVDESS/Aligned'
inverted_img_root = '/kuacc/users/abond19/VideoEditing/RAVDESS_inversion_prev/image_inversions'
inversion_root = '/kuacc/users/abond19/VideoEditing/RAVDESS_inversion_prev/w+_inversions'

In [3]:
# model_dir = '/kuacc/users/mali18/dicomogan/logs/tmp_log/38_k___541_fashion_selfattention2022-10-30T03-54-43/__541_fashion_selfattention2022-10-30T03-54-43'
# img_root = '/kuacc/users/abond19/datasets/aligned_fashion_dataset'
# inverted_img_root =  '/kuacc/users/abond19/datasets/inverted_fashion_dataset'
# inversion_root =  '/kuacc/users/abond19/datasets/w+_fashion_dataset/fashion/PTI/'

In [4]:
device = 'cuda'

In [14]:
torch.set_grad_enabled(False) 

In [5]:
from experiments_utils import *
model = load_model_from_dir(model_dir).to(device)

Using /scratch/users/mali18/.cache/torch_extensions/py38_cu113 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /scratch/users/mali18/.cache/torch_extensions/py38_cu113/fused/build.ninja...
Building extension module fused...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module fused...
Loading custom kernel...
Using /scratch/users/mali18/.cache/torch_extensions/py38_cu113 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /scratch/users/mali18/.cache/torch_extensions/py38_cu113/upfirdn2d/build.ninja...
Building extension module upfirdn2d...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module upfirdn2d...


Using cache found in /kuacc/users/mali18/.cache/torch/hub/facebookresearch_dino_main


Restored from /kuacc/users/mali18/dicomogan/logs/_510_face_splice2022-11-01T02-55-15/VideoManipulation/_510_face_splice2022-11-01T02-55-15/checkpoints/epoch=194-step=53819.ckpt


In [6]:
# load new fine-tuned generator
from dicomogan.models.stylegan import StyleGAN2Face
finetune_gen = StyleGAN2Face("/kuacc/users/abond19/VideoEditing/encoder4editing/updated_stylegan.pt")

In [ ]:
# cat faces


In [19]:
model.stylegan_G = finetune_gen.to(device)

In [8]:
n_start = 5
n_frames = 12

In [9]:
# load images
import os
import torch 
from PIL import Image
import torchvision.transforms as transforms
IMG_EXTENSIONS = ['.png', '.PNG']
TXT_EXTENSIONS = ['.txt']

crop = None
size = None
trans_list = []
if crop is not None:
    trans_list.append(transforms.CenterCrop(tuple(crop)))
if size is not None:
    trans_list.append(transforms.Resize(tuple(size)))
trans_list.append(transforms.ToTensor())
img_transform=transforms.Compose(trans_list)

def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

def is_text_file(filename):
    return any(filename.endswith(extension) for extension in TXT_EXTENSIONS)

def get_image(img_path):
    img = Image.open(img_path).convert('RGB')
    return img

def get_inversion(inversion_path):
    w_vector = torch.load(inversion_path, map_location='cpu')
    if len(w_vector.shape) == 2:
        w_vector = w_vector.unsqueeze(0)
    assert (w_vector.shape == (1, 18, 512)), "Inverted vector has incorrect shape"
    return w_vector

def load_video(vid_path):
    images, inversions, sampleT, inversion_imgs = [], [], [], [] 
    fname = vid_path
    
    lst_vid = sorted(os.listdir(os.path.join(img_root, fname)))
    def correct(f):
        if is_image_file(f):
            return int(f.split('.')[0])
        else:
            return f
    lst_vid = list(map(correct, lst_vid))[:-1]
#     print(lst_vid)
    inds = np.argsort(lst_vid)
#     print(inds)
    inds = inds[n_start:n_start+n_frames]
    for f in np.array(sorted(os.listdir(os.path.join(img_root, fname))))[inds]:
        if is_image_file(f):
#             print(f)
            imname = f[:-4]
            images.append(img_transform(get_image(os.path.join(img_root, fname, f))))
            inversion_imgs.append(img_transform(get_image(os.path.join(inverted_img_root, fname, f))))
            inversions.append(get_inversion(os.path.join(os.path.join(inversion_root, fname, imname + ".pt"))))
            sampleT.append(int(imname))
    
    with open(os.path.join(img_root, fname, 'updated_descriptions.txt')) as f:
        description = f.readlines()[0]

    return torch.stack(images).to(device), torch.cat(inversions, 0).to(device), torch.Tensor(sampleT).to(device), torch.stack(inversion_imgs).to(device), description

In [10]:
import imageio
import torchvision
def save_gif(video, range, save_path):
    # Assuming that the current shape is T * B x C x H x W
    with imageio.get_writer(save_path, mode='I') as writer:
       for b_frames in video:
            # b_frames B x C x H x W
            frame = torchvision.utils.make_grid(b_frames,
                            nrow=b_frames.shape[0],
                            normalize=True,
                            range=range).detach().cpu().numpy()
            frame = (np.transpose(frame, (1, 2, 0)) * 255).astype(np.uint8)
            writer.append_data(frame)

#     wandb.log({name: wandb.Video(filename, fps=2, format="gif")})
#     os.remove(filename)


In [11]:
videos = [
    '8c110571',
    '2c111960',
    '8c110147'
]

In [12]:
videos = [
    'Actor_03_114',
    'Actor_04_97',
    'Actor_05_99'
]

# Reconstruct Video

In [24]:
model.delta_inversion_weight = 0.03

In [25]:
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import torch.nn as nn
with torch.no_grad():
    for video in tqdm(videos):
        images, inversions, sampleT, inversion_imgs, src_desc = load_video(video)
        save_dir = os.path.join('applications_results', f"{model_dir.split('/')[-1]}", video)
        os.makedirs(save_dir, exist_ok=True)
        # org
        save_gif(images, (0, 1), f'{save_dir}/original.gif')
        
        # inversion
        save_gif(inversion_imgs, (0, 1), f'{save_dir}/inversion.gif')
        
        tgt_desc = "A picture of a woman"
        txt_feat = model.clip_encode_text([src_desc]).cuda()  # - model.clip_encode_text([src_desc])
        
        ind = np.random.randint(len(sampleT))
        frame_feat = model.clip_loss.encode_images(images[ind:ind+1])
        
        edited_videos = model(images.unsqueeze(0), 
                              sampleT,
                              inversions.mean(0, keepdims=True),
                              txt_feat,
                             frame_feat)[0]
        save_gif(edited_videos, (-1, 1), f'{save_dir}/Mean_Frame.gif')
        to_PIL(model.stylegan_G(inversions.mean(0, keepdims=True))[0]).save(f'{save_dir}/mean_frame.png')
        
        
        edited_videos = model(images.unsqueeze(0), 
                              sampleT,
                              inversions[-1:],
                              txt_feat,
                             frame_feat)[0]
        save_gif(edited_videos, (-1, 1), f'{save_dir}/Last_Frame.gif')
        to_PIL(model.stylegan_G(inversions[-1:])[0]).save(f'{save_dir}/last_frame.png')
        
        edited_videos = model(images.unsqueeze(0), 
                              sampleT,
                              inversions[0:1],
                              txt_feat,
                             frame_feat)[0]
        save_gif(edited_videos, (-1, 1), f'{save_dir}/First_Frame.gif')
        to_PIL(model.stylegan_G(inversions[0:1])[0]).save(f'{save_dir}/first_frame.png')
        
        
        mid = inversions.shape[0] // 2
        edited_videos = model(images.unsqueeze(0), 
                              sampleT,
                              inversions[mid:mid+1],
                              txt_feat,
                             frame_feat)[0]
        save_gif(edited_videos, (-1, 1), f'{save_dir}/Middle_Frame.gif')
        to_PIL(model.stylegan_G(inversions[mid:mid+1])[0]).save(f'{save_dir}/middle_frame.png')
#         edited_videos = forward(model, images.unsqueeze(0), inversions.unsqueeze(0), sampleT, [des], flag='L')[0]
#         save_gif(edited_videos.permute(0, 2, 3, 1).detach().cpu().numpy(), f'{save_dir}/Last_frame')

#         edited_videos = forward(model, images.unsqueeze(0), inversions.unsqueeze(0), sampleT, [des], flag='F')[0]
#         save_gif(edited_videos.permute(0, 2, 3, 1).detach().cpu().numpy(), f'{save_dir}/First_frame')

        
        

100%|██████████| 3/3 [01:14<00:00, 24.80s/it]


# Generate images for eval 

In [90]:
# train_path = 'data/fashion/fashion_train_videos.txt'
# test_path = 'data/fashion/fashion_test_videos.txt'

In [91]:
train_path = '/kuacc/users/mali18/dicomogan/data/face/action_train.txt'
test_path = '/kuacc/users/mali18/dicomogan/data/face/action_test.txt'

In [129]:
with open(train_path, 'r') as f:
    train_videos = f.read().split('\n')

In [130]:
with open(test_path, 'r') as f:
     test_videos = f.read().split('\n')

In [136]:
from tqdm import tqdm
def generate(lst, save_dir, bs=2, save_size=(256,256)):
    with torch.no_grad():
        for i in tqdm(range(0, len(lst), bs)):
            images, inversions, descriptions = [], [], []
            for j in range(i, i+bs):
                a, b, c, _, desc = load_video(lst[j])
                images.append(a)
                inversions.append(b)
                sampleT = c
                descriptions.append(desc)
            
            images, inversions = torch.stack(images, 0), torch.stack(inversions, 0)
            txt_feat = model.clip_encode_text(descriptions)  # - model.clip_encode_text([src_desc])

            ind = np.random.randint(len(sampleT))
            frame_feat = model.clip_loss.encode_images(images[:, ind])

#             print(images.shape, sampleT.shape, inversions.mean(1, keepdims=False).shape, txt_feat.shape, frame_feat.shape)
            edited_videos = model(images, 
                                  sampleT,
                                  inversions.mean(1, keepdims=False),
                                  txt_feat,
                                  frame_feat)

    # #             print(torch.stack(images, 0).shape, torch.stack(inversions, 0).shape)
    #             edited_videos = model(torch.stack(images, 0), torch.stack(inversions, 0), sampleT, [desc] * bs)          
            for video, video_name in zip(edited_videos, lst[i:i+bs]):
                save_path = os.path.join(save_dir, video_name)
                os.makedirs(save_path, exist_ok=True)
                for j, frame in enumerate(video):
                    to_PIL(frame).resize(save_size).save(os.path.join(save_path, f"{j:06d}.png"))

In [132]:
train_save_dir = os.path.join("model_outputs", f"{model_dir.split('/')[-1]}", "train")

In [133]:
test_save_dir = os.path.join("model_outputs", f"{model_dir.split('/')[-1]}", "test")

In [134]:
train_videos = np.array(train_videos)
test_videos = np.array(test_videos)

In [137]:
generate(train_videos[:200], train_save_dir)

100%|██████████| 100/100 [09:14<00:00,  5.54s/it]


In [26]:
generate(test_videos[:200], test_save_dir)

[autoreload of dicomogan.minigpt failed: Traceback (most recent call last):
  File "/kuacc/users/mali18/.conda/envs/styleode/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/kuacc/users/mali18/.conda/envs/styleode/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 480, in superreload
    update_generic(old_obj, new_obj)
  File "/kuacc/users/mali18/.conda/envs/styleode/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 377, in update_generic
    update(a, b)
  File "/kuacc/users/mali18/.conda/envs/styleode/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 345, in update_class
    update_instances(old, new)
  File "/kuacc/users/mali18/.conda/envs/styleode/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 299, in update_instances
    refs = gc.get_referrers(old)
KeyboardInterrupt
]

KeyboardInterrupt



In [96]:
Image.open('/kuacc/users/abond19/datasets/aligned_fashion_dataset/7c90376/00010.png').size

(512, 1024)